<h3>Determining Inference Using MultiNLI</h3>
<b>Term Paper</b>

<b>Chris Huber\
CS820\
Prof. Kulkari, Spring 2022\
</b> 

In [5]:
import numpy as np
import pandas as pd
import torch

In [6]:
# import the train for the MultiNLI dataset
json_train_file = "multinli_1.0/multinli_1.0_train.jsonl"
df_json = pd.read_json(json_train_file, lines=True)

ValueError: Expected object or value

In [4]:
# output head rows to check
df_json.head()

NameError: name 'df_json' is not defined

In [14]:
# import pretrained Roberta
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained('roberta.large', checkpoint_file='model.pt')
roberta.eval()  # disable dropout (or leave in train mode to finetune)

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwi

In [24]:
# Download RoBERTa already finetuned for MNLI
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()  # disable dropout for evaluation

Using cache found in /Users/chris/.cache/torch/hub/pytorch_fairseq_main
100%|██████████████████████████████████████████████████████| 751652118/751652118 [00:16<00:00, 45908425.20B/s]


RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwi

In [25]:
# Example code from https://github.com/pytorch/fairseq/tree/main/examples/roberta#use-roberta-for-sentence-pair-classification-tasks
# test inference
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
# roberta.cuda()
roberta.eval()
# with open('glue_data/MNLI/dev_matched.tsv') as fin:
with open('multinli_1.0/multinli_1.0_dev_matched.txt') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[8], tokens[9], tokens[-1]
        tokens = roberta.encode(sent1, sent2)
        prediction = roberta.predict('mnli', tokens).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.3123
